# Generative AI (CS F437) Assignment 1
Training and Evaluating Transformer Models for English to Hindi Translation

## Part 1: Fine-Tuning a Transformer Model

### Import Dependencies

In [2]:
!pip install --quiet --upgrade pip
!pip install numpy pandas matplotlib nltk wandb datasets transformers evaluate







  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 10.9 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 10.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.7 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Attempting unins

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import nltk
import wandb

from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, TrainingArguments, Trainer
import evaluate

# Download NLTK data required for BLEU computation
nltk.download('punkt', quiet=True)

We use WandB to store out weights and training data

In [ ]:
wandb_api_key = input("Enter your wandb API key: ") 
wandb.login(key=wandb_api_key)

# Initialize wandb run (change project and entity as desired)
wandb.init(
    project="GEN_AI",
    entity="aashreyrachaputi-bits-pilani", 
    config={
        "model_name": "Helsinki-NLP/opus-mt-en-hi",
        "dataset": "cfilt/iitb-english-hindi",
        "num_train_epochs": 3,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 16,
        "per_device_eval_batch_size": 16,
        "num_decoder_layers_to_keep_trainable": 1,
    }
)

### Data Loading and Preprocessing
Load the IITB English-Hindi parallel corpus and prepare it for training.

In [ ]:
# Load the dataset
ds = load_dataset("cfilt/iitb-english-hindi")
train_data = ds["train"]
val_data = ds["validation"]
test_data = ds["test"]

# Create text dictionaries for evaluation
train_texts = {
    "en": [example["en"] for example in train_data["translation"]],
    "hi": [example["hi"] for example in train_data["translation"]],
}
val_texts = {
    "en": [example["en"] for example in val_data["translation"]],
    "hi": [example["hi"] for example in val_data["translation"]],
}
test_texts = {
    "en": [example["en"] for example in test_data["translation"]],
    "hi": [example["hi"] for example in test_data["translation"]],
}

### Training Configuration
Set up the training arguments and trainer.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True, 
    report_to=["wandb"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

### Model Layer Freezing
Freeze specific layers in the model to focus training on the most important components.

In [ ]:
def freeze_layers(model, freeze_encoder=True, num_decoder_layers_to_keep_trainable=2):
    for param in model.parameters():
        param.requires_grad = False

    if not freeze_encoder:
        for param in model.model.encoder.embed_tokens.parameters():
            param.requires_grad = True
        for layer in model.model.encoder.layers:
            for param in layer.parameters():
                param.requires_grad = True

    # Unfreeze decoder embeddings
    for param in model.model.decoder.embed_tokens.parameters():
        param.requires_grad = True

    # Unfreeze the last N decoder layers only
    total_decoder_layers = len(model.model.decoder.layers)
    for i in range(total_decoder_layers - num_decoder_layers_to_keep_trainable, total_decoder_layers):
        if i >= 0:
            for param in model.model.decoder.layers[i].parameters():
                param.requires_grad = True

    # Always unfreeze the final output projection
    for param in model.lm_head.parameters():
        param.requires_grad = True

    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    wandb.log({
        "trainable_parameters": trainable_params,
        "total_parameters": total_params,
        "trainable_percentage": trainable_params/total_params
    })
    
    architecture_info = {
        "encoder_status": "Frozen" if freeze_encoder else "Trainable",
        "decoder_status": f"Partially trainable (last {num_decoder_layers_to_keep_trainable} layers)",
        "output_projection": "Trainable"
    }
    wandb.log({"model_architecture": architecture_info})

In [ ]:
freeze_layers(model, freeze_encoder=True, num_decoder_layers_to_keep_trainable=7)

In [ ]:
# Start training
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./pretrain_model_7l")
tokenizer.save_pretrained("./pretrain_model_7l")

In [ ]:
def translate(texts):
    batch_size = 8
    translations = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
        # Ensure tensors are on the correct device
        input_ids = inputs.input_ids.to(model.device)
        attention_mask = inputs.attention_mask.to(model.device)
        # Generate translations using beam search
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=5,
            early_stopping=True
        )
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        translations.extend(decoded_outputs)
    return translations

### Evaluation

Once we have trained the model and saved it to our hardrive, we can directly import it from there to test.

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch

model_dir = "./pretrain_model_7l"

# Load tokenizer from the folder
tokenizer = MarianTokenizer.from_pretrained(model_dir)

# Load the model from the folder
model = MarianMTModel.from_pretrained(model_dir)
model.eval()

#### Interactive Translation Demo

In [ ]:
# Simple translation loop
while True:
    english_text = input("Enter an English sentence (or 'exit' to quit): ")
    if english_text.strip().lower() == 'exit':
        break
    inputs = tokenizer(english_text, return_tensors="pt")
    with torch.no_grad():
        generated_tokens = model.generate(**inputs)
    hindi_translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    print("Hindi Translation:", hindi_translation)

#### Model Evaluation

Evaluate the translation model using standard metrics including:
- BLEU score: Measures n-gram overlap between translations and references
- ROUGE score: Measures recall of n-grams between translations and references
- METEOR score: Measures word-to-word matches between translations and references

In [ ]:
import torch
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import evaluate

nltk.download('punkt')

rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

dataset = load_dataset("cfilt/iitb-english-hindi", split="test")
source_texts = [item["translation"]["en"] for item in dataset]
reference_texts = [item["translation"]["hi"] for item in dataset]

print(len(source_texts))
print(len(reference_texts))

model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate(texts, batch_size=8):
    predictions = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                num_beams=5,
                max_length=128,
                early_stopping=True
            )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(decoded)
    return predictions

predictions = translate(source_texts)

references_tokenized = [[ref.split()] for ref in reference_texts]
predictions_tokenized = [pred.split() for pred in predictions]

# Compute BLEU score
smoothing_function = SmoothingFunction().method1
bleu_score = corpus_bleu(references_tokenized, predictions_tokenized, smoothing_function=smoothing_function)
print(f"Corpus BLEU Score: {bleu_score:.4f}")

# Compute ROUGE scores
rouge_results = rouge.compute(predictions=predictions, references=reference_texts)
print(f"ROUGE-1 F1 Score: {rouge_results['rouge1']:.4f}")
print(f"ROUGE-2 F1 Score: {rouge_results['rouge2']:.4f}")
print(f"ROUGE-L F1 Score: {rouge_results['rougeL']:.4f}")

# Compute METEOR score
meteor_score = meteor.compute(predictions=predictions, references=reference_texts)
print(f"METEOR Score: {meteor_score['meteor']:.4f}")

#### Visualize Results

The cell below can be used to visualize translation examples and quality metrics.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import random

num_examples = 20
random_indices = random.sample(range(len(source_texts)), num_examples)

examples = []
for idx in random_indices:
    examples.append({
        "English": source_texts[idx],
        "Hindi (Reference)": reference_texts[idx],
        "Hindi (Predicted)": predictions[idx]
    })

pd.DataFrame(examples)

## Part 2: Trained Transformer Model

### Environment Setup

In [ ]:
!pip install datasets evaluate transformers torch

In [ ]:
!pip install sentencepiece sacrebleu sacremoses nltk rouge-score

In [ ]:
!pip install optuna

### Loading Dataset and Model

We'll use the IITB English-Hindi parallel corpus and the mBART-50 multilingual translation model.

In [ ]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch


ds = load_dataset("cfilt/iitb-english-hindi")
test_data = ds["test"]

In [ ]:
# Load pre-trained mBART model and tokenizer
print("Loading model and tokenizer...")
model_name = "facebook/mbart-large-50"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set source and target languages
tokenizer.src_lang = "en_XX"  
tokenizer.tgt_lang = "hi_IN"
print("Model and tokenizer loaded successfully")

## 3. Model Evaluation

We'll evaluate the model using three metrics:

1. **BLEU** (Bilingual Evaluation Understudy): Measures precision of n-grams
2. **ROUGE** (Recall-Oriented Understudy for Gisting Evaluation): Recall-oriented metric for summarization
3. **METEOR** (Metric for Evaluation of Translation with Explicit ORdering): Harmonic mean of precision and recall

In [ ]:
from evaluate import load

def evaluate_model(model, dataset, max_samples=None):
    model.eval()
    metric_bleu = load("sacrebleu")
    metric_rouge = load("rouge")
    metric_meteor = load("meteor")
    
    references = []
    predictions = []
    
    eval_dataset = dataset[:max_samples] if max_samples else dataset
    total = len(eval_dataset)
    
    for i, example in enumerate(eval_dataset):
        print(f"Processing... {i+1}/{total} ({(i+1)/total*100:.1f}%)")
        
        input_text = example["translation"]["en"]
        ref_text = example["translation"]["hi"]
        
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids
        with torch.no_grad():
            outputs = model.generate(
                inputs, 
                forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
                max_length=128
            )
        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        references.append([ref_text])
        predictions.append(pred_text)
    
    bleu_score = metric_bleu.compute(predictions=predictions, references=references)
    rouge_score = metric_rouge.compute(predictions=predictions, references=references)
    meteor_score = metric_meteor.compute(predictions=predictions, references=references)
    
    return bleu_score, rouge_score, meteor_score

### Running the Evaluation

In [ ]:
MAX_SAMPLES = 1000 

bleu, rouge, meteor = evaluate_model(model, test_data, MAX_SAMPLES)

print("\n===== Evaluation Results =====\n")
print(f"BLEU Score: {bleu['score']:.2f}")
print(f"ROUGE-1 Precision: {rouge['rouge1'].mid.precision:.4f}, Recall: {rouge['rouge1'].mid.recall:.4f}, F1: {rouge['rouge1'].mid.fmeasure:.4f}")
print(f"ROUGE-2 F1: {rouge['rouge2'].mid.fmeasure:.4f}")
print(f"ROUGE-L F1: {rouge['rougeL'].mid.fmeasure:.4f}")
print(f"METEOR Score: {meteor['meteor']:.4f}")

### Sample Translations

In [ ]:
def show_sample_translations(model, dataset, num_samples=5):
    model.eval()
    import random
    samples = random.sample(range(len(dataset)), num_samples)
    
    for i, idx in enumerate(samples):
        example = dataset[idx]
        input_text = example["translation"]["en"]
        ref_text = example["translation"]["hi"]
        
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids
        with torch.no_grad():
            outputs = model.generate(inputs, forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        print(f"\nExample {i+1}:")
        print(f"English: {input_text}")
        print(f"Reference Hindi: {ref_text}")
        print(f"Predicted Hindi: {pred_text}")

show_sample_translations(model, test_data, 25)